# RAG with Multi Data Source

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki.name)

wikipedia


In [19]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings

from langchain_community.chat_models import ChatOllama



# llm = Ollama(model="llama3.2")
# embeddings = OllamaEmbeddings(model="llama3.2")
llm = ChatOllama(model="llama3.2")
embeddings = OllamaEmbeddings(model="llama3.2")



loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents, embeddings)
retriever=vectordb.as_retriever
retriever

C:\Users\arsal\AppData\Local\Temp\ipykernel_18916\4215999419.py:13: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.2")


<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x0000018E26B87E20>>

In [20]:
from langchain.tools.retriever import create_retriever_tool

retrieval_tool=create_retriever_tool(retriever,"langsmith_search", "Search for information about Langsmith")


In [21]:
retrieval_tool.name

'langsmith_search'

In [22]:
# Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [23]:
tools=[wiki,arxiv,retrieval_tool]

In [24]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\arsal\\OneDrive\\Desktop\\LANGCHAIN\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search for information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000018E17081090>, retriever=<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x0000018E26B87E20>>, document_pr

In [25]:
# Prompt Template
from langchain import hub

#get the prompt from hub
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [26]:
#Agents
from langchain.agents import create_openai_tools_agent
agents=create_openai_tools_agent(llm,tools,prompt)

In [27]:
agents

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [28]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agents,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [29]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
Langsmith is a company that specializes in providing AI-powered language tools and services for individuals, businesses, and organizations. They offer a range of products and solutions designed to help users improve their writing skills, generate content, and automate linguistic tasks.

Some of the key features of Langsmith include:

1. Writing analysis: Their tools can analyze text and provide feedback on clarity, grammar, syntax, and style.
2. Content generation: Langsmith offers AI-powered content generation capabilities, allowing users to create content quickly and efficiently.
3. Language translation: They provide language translation services for individuals and businesses looking to expand their reach into new markets or communicate with clients in different languages.
4. Editing and proofreading: Their tools can help identify grammatical errors, punctuation mistakes, and other issues that may affect the quality of written content.

Langsm

{'input': 'Tell me about Langsmith',
 'output': "Langsmith is a company that specializes in providing AI-powered language tools and services for individuals, businesses, and organizations. They offer a range of products and solutions designed to help users improve their writing skills, generate content, and automate linguistic tasks.\n\nSome of the key features of Langsmith include:\n\n1. Writing analysis: Their tools can analyze text and provide feedback on clarity, grammar, syntax, and style.\n2. Content generation: Langsmith offers AI-powered content generation capabilities, allowing users to create content quickly and efficiently.\n3. Language translation: They provide language translation services for individuals and businesses looking to expand their reach into new markets or communicate with clients in different languages.\n4. Editing and proofreading: Their tools can help identify grammatical errors, punctuation mistakes, and other issues that may affect the quality of written 

In [32]:
agent_executor.invoke({"input":"What's the paper 2005.14165 about?"})



> Entering new AgentExecutor chain...
I couldn't find any information on a specific paper with the identifier "2005.14165". It's possible that it's a private or internal document, or it may not be a well-known or published work.

If you could provide more context or details about the paper, such as its title, authors, or field of study, I may be able to help you better. Alternatively, if you're looking for information on a specific topic or subject, feel free to ask and I'll do my best to assist you!

> Finished chain.


{'input': "What's the paper 2005.14165 about?",
 'output': 'I couldn\'t find any information on a specific paper with the identifier "2005.14165". It\'s possible that it\'s a private or internal document, or it may not be a well-known or published work.\n\nIf you could provide more context or details about the paper, such as its title, authors, or field of study, I may be able to help you better. Alternatively, if you\'re looking for information on a specific topic or subject, feel free to ask and I\'ll do my best to assist you!'}